## Other scrapers

Notes: Be careful when re-run this script as many face images are replaced with current office holder 
on relevant official websites

In [1]:
import pandas as pd
import bs4 as bs 
import urllib.request
import requests
import re 

Load wmpids 

In [2]:
person = pd.read_csv("../datasets/people/person_2022.csv")

### International leaders

#### G20 

The following code is simple but scrape smaller thumb images; instead, we use larger profile
images from their personal pages for better face recognition results

In [3]:
# url = 'https://en.wikipedia.org/wiki/G20'
# source = req.urlopen(url).read()
# soup = bs.BeautifulSoup(source)

# image_css = '.gallerybox .image img'
# images = soup.select(image_css)

# images_src = [i['src'][2:] for i in images]
# print(images_src)

# df = pd.DataFrame()
# df['url'] = images_src

# df.head()

In [4]:
#Download the website's html
source = urllib.request.urlopen("https://en.wikipedia.org/wiki/G20").read()
soup = bs.BeautifulSoup(source)
urls_css = ".center p a"
urls = soup.select(urls_css)

In [5]:
l1 = urls[1:14:3]
l2 = urls[17::3]
leaders = l1+l2

In [6]:
l_names = [url.text for url in leaders]

In [7]:
extracted_urls = [url["href"] for url in urls]

In [8]:
w1 = extracted_urls[1:14:3]
w2 = extracted_urls[17::3]
pages = w1+w2

In [9]:
page_urls = ["https://en.wikipedia.org/" + url_suffix for url_suffix in pages]

In [10]:
g20 = []

for i in page_urls[:]:
  source = urllib.request.urlopen(i).read()
  soup = bs.BeautifulSoup(source)
  img_css = ".infobox-image a img"  
  img = soup.select(img_css)
  g20.extend(list(set([i['src'] for i in img])))

In [11]:
g20_face_urls = ["https:" + url_suffix for url_suffix in g20]

In [12]:
len(g20_face_urls)

25

In [13]:
t = pd.DataFrame(g20_face_urls)

In [14]:
t = t.loc[t[0].str.contains("220px")]

In [15]:
l_urls = t[0].to_list()

In [16]:
len(l_urls)

23

In [17]:
l_urls.pop(-1)

'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Ursula_Von_der_Leyen_-_April_2022.jpg/220px-Ursula_Von_der_Leyen_-_April_2022.jpg'

In [18]:
l_urls.pop(16)

'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/President_of_South_Africa_MC_Ramaphosa_speaks_to_Russian_President_%2828_June_2019%29.jpg/220px-President_of_South_Africa_MC_Ramaphosa_speaks_to_Russian_President_%2828_June_2019%29.jpg'

In [19]:
g20 = pd.DataFrame()
g20['face_name']=l_names
g20['face_url']=l_urls

In [20]:
# To make name consistent with names in the person file:
g20.loc[g20['face_name']=='Salman', 'face_name'] = 'Mohammed bin Salman'
# Replace the president of Brazil
g20.loc[g20['face_name']=='Luiz Inácio Lula da Silva', 'face_name'] = 'Jair Bolsonaro'
g20.loc[g20['face_name']=='Luiz Inácio Lula da Silva', 'face_url'] = 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Jair_Bolsonaro_em_24_de_abril_de_2019_%281%29_%28cropped%29.jpg/220px-Jair_Bolsonaro_em_24_de_abril_de_2019_%281%29_%28cropped%29.jpg'

In [21]:
g20.shape

(21, 2)

In [22]:
l1 = {'face_name': 'António Guterres', 'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d1/Ant%C3%B3nio_Guterres_2021.jpg/220px-Ant%C3%B3nio_Guterres_2021.jpg"}
il = g20.append(l1, ignore_index = True)

In [23]:
il['face_category']  = 'IntLeader'

In [24]:
il.shape

(22, 3)

Get wmp ids

In [25]:
wmp_il = person.loc[person['intl_leaders']==1]

In [26]:
wmp_il.shape

(22, 30)

In [27]:
wmp_il = wmp_il[['full_name', 'wmpid']]

In [28]:
il = il.merge(wmp_il, how='left', left_on='face_name', right_on='full_name', indicator=True)

In [29]:
il.loc[il['face_name']=='Joe Biden', 'wmpid']='WMPID1289'

In [30]:
il.loc[il._merge =='left_only']

,face_name,face_url,face_category,full_name,wmpid,_merge
18,Joe Biden,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,NaN,WMPID1289,left_only


In [31]:
il

,face_name,face_url,face_category,full_name,wmpid,_merge
0,Alberto Fernández,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Alberto Fernández,WMPID5195,both
1,Anthony Albanese,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Anthony Albanese,WMPID5217,both
2,Jair Bolsonaro,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Jair Bolsonaro,WMPID5182,both
3,Justin Trudeau,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Justin Trudeau,WMPID5261,both
4,Xi Jinping,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Xi Jinping,WMPID5255,both
5,Emmanuel Macron,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Emmanuel Macron,WMPID5260,both
6,Olaf Scholz,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Olaf Scholz,WMPID5173,both
7,Narendra Modi,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Narendra Modi,WMPID5258,both
8,Joko Widodo,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Joko Widodo,WMPID5249,both
9,Giorgia Meloni,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Giorgia Meloni,WMPID5188,both


In [32]:
il = il[['wmpid', 'face_url', 'face_category']]

In [33]:
il.shape

(22, 3)

In [34]:
il.head(2)

,wmpid,face_url,face_category
0,WMPID5195,https://upload.wikimedia.org/wikipedia/commons...,IntLeader
1,WMPID5217,https://upload.wikimedia.org/wikipedia/commons...,IntLeader


### Former presidents

In [35]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States")
soup = bs.BeautifulSoup(res.content, "html.parser")

In [36]:
names =[]

for name in soup.find_all('b')[0:46]:
    names.append(name.get_text())

In [37]:
urls=[]
for link in soup.find_all('img')[6:52]:
    urls.append(link.get('src'))

In [38]:
urls = ['https:' + str(i).replace('160px', '320px') for i in urls]

In [39]:
fp = pd.DataFrame(columns=('face_name', 'face_url'))

In [40]:
fp['face_name']=names

In [41]:
fp['face_url']=urls

In [42]:
fp.shape

(46, 2)

In [43]:
fp.head()

,face_name,face_url
0,George Washington,https://upload.wikimedia.org/wikipedia/commons...
1,John Adams,https://upload.wikimedia.org/wikipedia/commons...
2,Thomas Jefferson,https://upload.wikimedia.org/wikipedia/commons...
3,James Madison,https://upload.wikimedia.org/wikipedia/commons...
4,James Monroe,https://upload.wikimedia.org/wikipedia/commons...


In [44]:
wmp_fp= person.loc[person['former_uspres']==1]
wmp_fp =wmp_fp[['wmpid', 'full_name']]

In [45]:
fp = fp.merge(wmp_fp, left_on='face_name', right_on='full_name', how='left')

In [46]:
fp.loc[fp.wmpid.isna()]

,face_name,face_url,wmpid,full_name
19,James A. Garfield,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN
36,Richard Nixon,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN
37,Gerald Ford,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN
38,Jimmy Carter,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN
42,George W. Bush,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN
43,Barack Obama,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN


In [47]:
wmp_fp.loc[wmp_fp['full_name'].str.contains("Carter")]

,wmpid,full_name
2122,WMPID5180,James Carter


In [48]:
fp.loc[fp['face_name']=='James A. Garfield', 'wmpid'] ='WMPID5184'
fp.loc[fp['face_name']=='George W. Bush', 'wmpid'] ='WMPID5'
fp.loc[fp['face_name']=='Barack Obama', 'wmpid'] ='WMPID3'
fp.loc[fp['face_name']=='Richard Nixon', 'wmpid'] ='WMPID5196'
fp.loc[fp['face_name']=='Gerald Ford', 'wmpid'] ='WMPID5197'
fp.loc[fp['face_name']=='Jimmy Carter', 'wmpid'] ='WMPID5180'

In [49]:
fp['face_category']='FormerPres'

Drop duplicated Grover Cleveland

In [50]:
fp = fp.drop_duplicates(keep='last')

In [51]:
fp = fp[['wmpid', 'face_url', 'face_category']]

In [52]:
fp.head(2)

,wmpid,face_url,face_category
0,WMPID5206,https://upload.wikimedia.org/wikipedia/commons...,FormerPres
1,WMPID5241,https://upload.wikimedia.org/wikipedia/commons...,FormerPres


In [53]:
fp.shape

(45, 3)

### Supreme Court Justices

All justices who took judicial oath after 1980

In [54]:
source = urllib.request.urlopen("https://www.supremecourt.gov/about/biographies.aspx").read()
soup = bs.BeautifulSoup(source)

In [55]:
urls_css = ".media-object"

In [56]:
urls = soup.select(urls_css)

In [57]:
names = [url["alt"] for url in urls]

In [58]:
urls = [url["src"] for url in urls]

In [59]:
urls = ["https://www.supremecourt.gov/about/" + url_suffix for url_suffix in urls]

In [60]:
s = pd.DataFrame()
s['face_name'] = names
s['face_url']=urls

In [61]:
n = s['face_name'].str.split(pat = ',', expand = True)

In [62]:
s['face_name'] = n[0]

In [63]:
s2 = {'face_name': 'Ruth Bader Ginsburg', 'face_url': "https://www.supremecourt.gov/about/justice_pictures/Official_Justice_Ginsburg_sm.jpg"}
s = s.append(s2, ignore_index = True)

In [64]:
s3 = {'face_name': 'William Rehnquist', 
      'face_url':'https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/William_Rehnquist.jpg/220px-William_Rehnquist.jpg'}
s = s.append(s3, ignore_index = True)

In [65]:
s4 = {'face_name': 'Antonin Scalia', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c5/Antonin_Scalia_Official_SCOTUS_Portrait.jpg/220px-Antonin_Scalia_Official_SCOTUS_Portrait.jpg"}
s = s.append(s4, ignore_index = True)

In [66]:
s.shape

(16, 2)

In [67]:
s.loc[s['face_name']=="Sandra Day O'Connor (Retired)", 'face_name']="Sandra Day O'Connor"

Get wmpid

In [68]:
wmp_s = person[['full_name', 'wmpid']]

In [69]:
s = s.merge(wmp_s, left_on='face_name', right_on='full_name', how='left')

In [70]:
s.loc[s['face_name']=="John G. Roberts", 'wmpid'] ='WMPID10'
s.loc[s['face_name']=="Samuel A. Alito", 'wmpid'] ='WMPID9'
s.loc[s['face_name']=="Neil M. Gorsuch", 'wmpid'] ='WMPID15'
s.loc[s['face_name']=="Brett M. Kavanaugh", 'wmpid'] ='WMPID16'
s.loc[s['face_name']=="Amy Coney Barrett", 'wmpid'] ='WMPID17'
s.loc[s['face_name']=="Stephen G. Breyer", 'wmpid'] ='WMPID12'
s.loc[s['face_name']=="Ruth Bader Ginsburg", 'wmpid'] ='WMPID18'

In [71]:
s['face_category']='SupremeCourt'

In [72]:
s

,face_name,face_url,full_name,wmpid,face_category
0,John G. Roberts,https://www.supremecourt.gov/about/justice_pic...,NaN,WMPID10,SupremeCourt
1,Clarence Thomas,https://www.supremecourt.gov/about/justice_pic...,Clarence Thomas,WMPID11,SupremeCourt
2,Samuel A. Alito,https://www.supremecourt.gov/about/justice_pic...,NaN,WMPID9,SupremeCourt
3,Sonia Sotomayor,https://www.supremecourt.gov/about/justice_pic...,Sonia Sotomayor,WMPID14,SupremeCourt
4,Elena Kagan,https://www.supremecourt.gov/about/justice_pic...,Elena Kagan,WMPID13,SupremeCourt
5,Neil M. Gorsuch,https://www.supremecourt.gov/about/justice_pic...,NaN,WMPID15,SupremeCourt
6,Brett M. Kavanaugh,https://www.supremecourt.gov/about/justice_pic...,NaN,WMPID16,SupremeCourt
7,Amy Coney Barrett,https://www.supremecourt.gov/about/justice_pic...,NaN,WMPID17,SupremeCourt
8,Ketanji Brown Jackson,https://www.supremecourt.gov/about/justice_pic...,Ketanji Brown Jackson,WMPID5166,SupremeCourt
9,Sandra Day O'Connor,https://www.supremecourt.gov/about/justice_pic...,Sandra Day O'Connor,WMPID5252,SupremeCourt


In [73]:
s = s[['wmpid', 'face_url', 'face_category']]

In [74]:
s.head(2)

,wmpid,face_url,face_category
0,WMPID10,https://www.supremecourt.gov/about/justice_pic...,SupremeCourt
1,WMPID11,https://www.supremecourt.gov/about/justice_pic...,SupremeCourt


### Cabinet

In [75]:
source = urllib.request.urlopen("https://www.whitehouse.gov/administration/cabinet/").read()
soup = bs.BeautifulSoup(source)

In [76]:
urls_css = ".persongrid__item__inner__content .h4alt"

In [77]:
urls = soup.select(urls_css)

In [78]:
c_names = [url.text for url in urls]

In [79]:
cab_names = []
for sub in c_names:
    cab_names.append(sub.replace("\n", "").replace("\t", "").replace("Dr. ", "")), 

In [80]:
urls_css = ".persongrid__item__inner__image>div img"

In [81]:
urls = soup.select(urls_css)

In [82]:
urls = [url["src"] for url in urls]

In [83]:
len(urls)

25

In [84]:
c = pd.DataFrame()
c['face_name'] = cab_names
c['face_url']=urls

In [85]:
# Replace chief of staff Jess Zients with his predecessor Ron Klain in 2022
c.loc[c['face_name']=='Jeff Zients', 'face_name']='Ron Klain'
c.loc[c['face_name']=='Jeff Zients', 'face_url']='https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Ron_Klain%2C_White_House_Chief_of_Staff.jpg/220px-Ron_Klain%2C_White_House_Chief_of_Staff.jpg'

Get wmpid

In [86]:
wmp = person[['full_name', 'wmpid']]

In [87]:
c = c.merge(wmp, left_on='face_name', right_on='full_name', how='left')

In [88]:
c.loc[c['wmpid'].isna()]

,face_name,face_url,full_name,wmpid
5,Deb Haaland,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
10,Marcia Fudge,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN
22,Shalanda Young,https://www.whitehouse.gov/wp-content/uploads/...,NaN,NaN


In [89]:
c.loc[c['face_name']=="Deb Haaland", 'wmpid'] ='WMPID1183'
c.loc[c['face_name']=="Marcia Fudge", 'wmpid'] ='WMPID1206'
c.loc[c['face_name']=="Shalanda Young ", 'wmpid'] ='WMPID5185'

In [90]:
c['face_category']='Cabinet'

In [91]:
c

,face_name,face_url,full_name,wmpid,face_category
0,Kamala Harris,https://www.whitehouse.gov/wp-content/uploads/...,Kamala Harris,WMPID2,Cabinet
1,Antony Blinken,https://www.whitehouse.gov/wp-content/uploads/...,Antony Blinken,WMPID5223,Cabinet
2,Janet Yellen,https://www.whitehouse.gov/wp-content/uploads/...,Janet Yellen,WMPID5214,Cabinet
3,Lloyd Austin,https://www.whitehouse.gov/wp-content/uploads/...,Lloyd Austin,WMPID5228,Cabinet
4,Merrick Garland,https://www.whitehouse.gov/wp-content/uploads/...,Merrick Garland,WMPID20,Cabinet
5,Deb Haaland,https://www.whitehouse.gov/wp-content/uploads/...,NaN,WMPID1183,Cabinet
6,Tom Vilsack,https://www.whitehouse.gov/wp-content/uploads/...,Tom Vilsack,WMPID5237,Cabinet
7,Gina Raimondo,https://www.whitehouse.gov/wp-content/uploads/...,Gina Raimondo,WMPID5198,Cabinet
8,Marty Walsh,https://www.whitehouse.gov/wp-content/uploads/...,Marty Walsh,WMPID5234,Cabinet
9,Xavier Becerra,https://www.whitehouse.gov/wp-content/uploads/...,Xavier Becerra,WMPID5236,Cabinet


In [92]:
c = c[['wmpid', 'face_url', 'face_category']]

In [93]:
c.head(2)

,wmpid,face_url,face_category
0,WMPID2,https://www.whitehouse.gov/wp-content/uploads/...,Cabinet
1,WMPID5223,https://www.whitehouse.gov/wp-content/uploads/...,Cabinet


### Public health related

In [94]:
ph = pd.DataFrame()

In [95]:
ph1 = {'face_name': 'Anthony Fauci', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/6/61/Anthony_Fauci_2020.jpg/220px-Anthony_Fauci_2020.jpg"}
ph = ph.append(ph1, ignore_index = True)

In [96]:
ph2 = {'face_name': 'Tedros Adhanom Ghebreyesus', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/Mukhisa_Kituyi%2C_Houlin_Zhao%2C_Tedros_Adhanom_Ghebreyesus_with_Sophia_-_AI_for_Good_Global_Summit_2018_%2841223188035%29_%28cropped%29.jpg/220px-Mukhisa_Kituyi%2C_Houlin_Zhao%2C_Tedros_Adhanom_Ghebreyesus_with_Sophia_-_AI_for_Good_Global_Summit_2018_%2841223188035%29_%28cropped%29.jpg"}
ph = ph.append(ph2, ignore_index = True)

In [97]:
#The Secretary of the Department of Health and Human Services
ph3 = {'face_name': 'Rachel L. Levine', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/2/28/Admiral_Rachel_L._Levine.jpg/220px-Admiral_Rachel_L._Levine.jpg"}
ph = ph.append(ph3, ignore_index = True)

In [98]:
#The Surgeon General
ph4 = {'face_name': 'Vivek H. Murthy', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/Vivek_Murthy%2C_Surgeon_General_%28profile%29.jpg/220px-Vivek_Murthy%2C_Surgeon_General_%28profile%29.jpg"}
ph = ph.append(ph4, ignore_index = True)

In [99]:
# Director of CDC
ph5 = {'face_name': 'Rochelle Walensky', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/b/b9/Rochelle_Walensky%2C_CDC_Director_%28cropped%29.jpg/220px-Rochelle_Walensky%2C_CDC_Director_%28cropped%29.jpg"}
ph = ph.append(ph5, ignore_index = True)

In [100]:
# Director of NIH
ph6 = {'face_name': 'Lawrence A. Tabak', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a3/Lawrence_A._Tabak_official_portrait.jpg/220px-Lawrence_A._Tabak_official_portrait.jpg"}
ph = ph.append(ph6, ignore_index = True)

In [101]:
# Director of FDA
ph7 = {'face_name': 'Robert Califf', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/5/51/Robert_M._Califf%2C_FDA_Commissioner.jpg/220px-Robert_M._Califf%2C_FDA_Commissioner.jpg"}
ph = ph.append(ph7, ignore_index = True)

In [102]:
ph['face_category']='PublicHealth'

In [103]:
ph

,face_name,face_url,face_category
0,Anthony Fauci,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
1,Tedros Adhanom Ghebreyesus,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
2,Rachel L. Levine,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
3,Vivek H. Murthy,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
4,Rochelle Walensky,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
5,Lawrence A. Tabak,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
6,Robert Califf,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth


In [104]:
ph.loc[ph['face_name']=='Tedros Adhanom Ghebreyesus', 'face_category']='IntLeader'

get wmpid

In [105]:
ph = ph.merge(wmp, left_on='face_name', right_on='full_name', how='left')

In [106]:
ph

,face_name,face_url,face_category,full_name,wmpid
0,Anthony Fauci,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Anthony Fauci,WMPID5215
1,Tedros Adhanom Ghebreyesus,https://upload.wikimedia.org/wikipedia/commons...,IntLeader,Tedros Adhanom Ghebreyesus,WMPID5242
2,Rachel L. Levine,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Rachel L. Levine,WMPID5207
3,Vivek H. Murthy,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Vivek H. Murthy,WMPID5216
4,Rochelle Walensky,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Rochelle Walensky,WMPID5199
5,Lawrence A. Tabak,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Lawrence A. Tabak,WMPID5247
6,Robert Califf,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth,Robert Califf,WMPID5175


In [107]:
ph = ph[['wmpid', 'face_url', 'face_category']]

In [108]:
ph.head(2)

,wmpid,face_url,face_category
0,WMPID5215,https://upload.wikimedia.org/wikipedia/commons...,PublicHealth
1,WMPID5242,https://upload.wikimedia.org/wikipedia/commons...,IntLeader


### Historical figures

In [109]:
h = pd.DataFrame()

In [110]:
h1 = {'face_name': 'Martin Luther King Jr.', 
      'face_url': "https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Martin_Luther_King%2C_Jr..jpg/220px-Martin_Luther_King%2C_Jr..jpg",
     'face_category': 'HistFigure'}
h = h.append(h1, ignore_index = True)

In [111]:
wmp.loc[wmp['full_name'].str.contains('Martin Luther')]

,full_name,wmpid
3458,Martin Luther King Jr.,WMPID5221


In [112]:
h['wmpid']='WMPID5221'

In [113]:
h

,face_category,face_name,face_url,wmpid
0,HistFigure,Martin Luther King Jr.,https://upload.wikimedia.org/wikipedia/commons...,WMPID5221


In [114]:
h = h[['wmpid', 'face_url', 'face_category']]

In [115]:
h.head()

,wmpid,face_url,face_category
0,WMPID5221,https://upload.wikimedia.org/wikipedia/commons...,HistFigure


### Combine all results

In [116]:
df = pd.concat([il, fp, s, c, ph, h])

In [117]:
df.shape

(116, 3)

In [118]:
df.head()

,wmpid,face_url,face_category
0,WMPID5195,https://upload.wikimedia.org/wikipedia/commons...,IntLeader
1,WMPID5217,https://upload.wikimedia.org/wikipedia/commons...,IntLeader
2,WMPID5182,https://upload.wikimedia.org/wikipedia/commons...,IntLeader
3,WMPID5261,https://upload.wikimedia.org/wikipedia/commons...,IntLeader
4,WMPID5255,https://upload.wikimedia.org/wikipedia/commons...,IntLeader


In [119]:
df['face_category'].value_counts()

FormerPres      45
Cabinet         25
IntLeader       23
SupremeCourt    16
PublicHealth     6
HistFigure       1
Name: face_category, dtype: int64

In [120]:
df.to_csv("./data/other_face_urls_2022.csv", index=False)